In [ ]:
!pip install -U /kaggle/input/bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl -qq

# LLM inference

When you develop your prompts, I advice you make inference from a much faster device.

However, you do need to make sure that the model outputs will be the same.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig

MODEL_PATH = "/kaggle/input/mixtral/pytorch/8x7b-instruct-v0.1-hf/1"

quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

config = AutoConfig.from_pretrained(MODEL_PATH)
config.gradient_checkpointing = True
config.do_sample = True

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map = "auto",
    trust_remote_code = True,
    quantization_config=quantization_config,
    config=config
)

In [ ]:
import textwrap
import transformers


def make_inference(instruction):
    instruction_format = "[INST]{instruction}[/INST]"
    text = instruction_format.format(instruction=instruction)
    inputs = tokenizer(text, return_tensors="pt").to('cuda')

    outputs = model.generate(**inputs, max_new_tokens=2000)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return output_text

In [ ]:
instruction = """
Write Python code to solve this problem in a brute force manner. You will only print the final result.

Find the largest two digit prime number which is also a prime number when the digits are reversed.

Reminder: You will write Python code. You do not care about the efficiency of the solution, you will only care about the correctness.
""".strip()

In [ ]:
output_text = make_inference(instruction)

In [ ]:
print(output_text)

# Code execution

In [ ]:
import re
def extract_code(reply):
    pattern = r"```python([\s\S]*?)```"
    matches = re.findall(pattern, reply)
    code = "\n\n".join(matches)
    if code:
        return code
    pattern = r"```([\s\S]*?)```"
    matches = re.findall(pattern, reply)
    code = "\n\n".join(matches)
    return code

In [ ]:
import sys
import io
import contextlib
import signal

def run_python_code(code, time_limit_seconds=20):
    @contextlib.contextmanager
    def time_limit(seconds):
        def signal_handler(signum, frame):
            raise TimeoutError("Timed out!")
        signal.signal(signal.SIGALRM, signal_handler)
        signal.alarm(seconds)
        try:
            yield
        finally:
            signal.alarm(0)

    old_stdout = sys.stdout
    redirected_output = sys.stdout = io.StringIO()

    try:
        with time_limit(time_limit_seconds):
            exec(code)
    except TimeoutError as e:
        print("Code execution timed out")
    except Exception as e:
        print(f"Error: {str(e)}")
    finally:
        sys.stdout = old_stdout

    return redirected_output.getvalue()

In [ ]:
execution_output = run_python_code("""
print(1+3)
""", time_limit_seconds=5)

print(execution_output)

In [ ]:
execution_output = run_python_code("""
import time
for i in range(10):
    print(i)
    time.sleep(1)
""", time_limit_seconds=5)

print(execution_output)

# Produce answers

In [ ]:
import pandas as pd
df = pd.read_csv("/kaggle/input/ai-mathematical-olympiad-prize/test.csv")

In [ ]:
from collections import defaultdict

instruction_template = """
Write Python code to solve this problem in a brute force manner. You will only print the final result.

{problem}

Reminder:
- You will write Python code.
- You will only attempt to solve the problem with brute force.
- You will only print the final result.
""".strip()

answers = []

for problem in df["problem"]:
    print(problem)
    responses = defaultdict(int)
    
    for _ in range(3):
        instruction = instruction_template.format(problem=problem)
        output_text = make_inference(instruction)
        print(f"{output_text=}")
        code = extract_code(output_text)
        print(f"{code=}")
        execution_output = run_python_code(code)
        print(f"{execution_output=}")
        try:
            result = int(execution_output)
            responses[result] += 1
        except ValueError:
            continue
        
    if len(responses) == 0:
        answer = 1
        answers.append(answer)  # random guess
    else:
        answer = sorted([(v,k) for k,v in responses.items()])[-1][-1]
        answers.append(answer%1000)  # most common guess

df["answer"] = answers

In [ ]:
print(code)

In [ ]:
df[["id", "answer"]].to_csv("submission.csv", index=False)

In [ ]:
!head /kaggle/input/ai-mathematical-olympiad-prize/sample_submission.csv

In [ ]:
!head submission.csv